In [ ]:
import json
import pickle
import numpy as np
import torch
import clip

model, _ = clip.load('RN50', device='cuda')

In [2]:
with open('vg/object_synsets.json', 'r') as file:
    objects = json.load(file)
with open('vg/attribute_synsets.json', 'r') as file:
    attributes = json.load(file)
with open('vg/synsets.json', 'r') as file:
    synsets = json.load(file)

vg_concepts = {}
for c in objects.values():
    if (c.split(".")[1] == 'n' or c.split(".")[1] == 'a') and len(c.split(".")[0]) > 2:
        cpt = c.split(".")[0].replace("_", " ")
        if cpt in vg_concepts:
            vg_concepts[cpt] += 1
        else:
            vg_concepts[cpt] = 1

for c in attributes.values():
    if (c.split(".")[1] == 'n' or c.split(".")[1] == 'a') and len(c.split(".")[0]) > 2:
        cpt = c.split(".")[0].replace("_", " ")
        if cpt in vg_concepts:
            vg_concepts[cpt] += 1
        else:
            vg_concepts[cpt] = 1

for c in synsets:
    if (c['synset_name'].split(".")[1] == 'n' or c['synset_name'].split(".")[1] == 'a') and len(c['synset_name'].split(".")[0]) > 2:
        cpt = c['synset_name'].split(".")[0].replace("_", " ")
        if cpt in vg_concepts:
            vg_concepts[cpt] += 1
        else:
            vg_concepts[cpt] = 1

In [43]:
threshold = 5
concepts = []

for cpt, times in vg_concepts.items():
    if times >= threshold:
        concepts.append(cpt)

In [44]:
concept_bank = {}
concept_length = 0

with torch.no_grad():
    for cpt in concepts:
        text = clip.tokenize(cpt).cuda()
        text_features = model.encode_text(text).cpu().numpy()
        text_features = text_features / np.linalg.norm(text_features)
        concept_bank[cpt] = text_features[0]
        concept_length += len(cpt)

concept_length = int(concept_length / len(concepts)) + 1

In [45]:
with open(f'vg/vg_num_{len(concept_bank)}_len_{concept_length}.pkl', 'wb') as file:
    pickle.dump(concept_bank, file)